In [1]:
import numpy as np

# import Rectangular bivariate spline from scipy
from scipy.interpolate import RegularGridInterpolator as RGI

def _orient_vectorfield(X, Y, Z, x, vector_field):
            
        # Check for orientational discontinuity by introducing appropriate scaling
        idx_x = np.searchsorted(X[0, :, 0], x[0])
        idx_y = np.searchsorted(Y[:, 0, 0], x[1])
        idx_z = np.searchsorted(Z[0, 0, :], x[2])
        
        if idx_x == 0:
            idx_x += 1
        if idx_y == 0:
            idx_y += 1
        if idx_z == 0:
            idx_z += 1   
    
        X_reduced = X[idx_y-1:idx_y+1, idx_x-1:idx_x+1, idx_z-1:idx_z+1]
        Y_reduced = Y[idx_y-1:idx_y+1, idx_x-1:idx_x+1, idx_z-1:idx_z+1]
        Z_reduced = Z[idx_y-1:idx_y+1, idx_x-1:idx_x+1, idx_z-1:idx_z+1]
    
        v_grid = np.zeros((2, 2, 2, 3))*np.nan
        for l in range(v_grid.shape[3]):
            v_grid[0,0,0,l] = vector_field[idx_y-1,idx_x-1, idx_z-1, l]
            v_grid[1,0,0,l] = vector_field[idx_y,idx_x-1, idx_z-1, l]
            v_grid[0,1,0,l] = vector_field[idx_y-1,idx_x, idx_z-1, l]
            v_grid[1,1,0,l] = vector_field[idx_y,idx_x, idx_z-1, l]
            v_grid[0,0,1,l] = vector_field[idx_y-1,idx_x-1, idx_z, l]
            v_grid[1,0,1,l] = vector_field[idx_y,idx_x-1, idx_z, l]
            v_grid[0,1,1,l] = vector_field[idx_y-1,idx_x, idx_z, l]
            v_grid[1,1,1,l] = vector_field[idx_y,idx_x, idx_z, l]
            
        vx_grid = v_grid[:,:,:,0]
        vy_grid = v_grid[:,:,:,1]
        vz_grid = v_grid[:,:,:,2]
            
        for i in range(2):
            for j in range(2):
                for k in range(2):
                    if vx_grid[0, 0, 0]*vx_grid[i, j, k]+vy_grid[0, 0, 0]*vy_grid[i, j, k]+vz_grid[0, 0, 0]*vz_grid[i, j, k] < 0:
                        vx_grid[i, j, k] = -vx_grid[i, j, k]
                        vy_grid[i, j, k] = -vy_grid[i, j, k]
                        vz_grid[i, j, k] = -vz_grid[i, j, k]
    
        vx_Interp = RGI((Y_reduced[:, 0, 0], X_reduced[0, :, 0], Z_reduced[0, 0, :]), vx_grid)
        vy_Interp = RGI((Y_reduced[:, 0, 0], X_reduced[0, :, 0], Z_reduced[0, 0, :]), vy_grid)
        vz_Interp = RGI((Y_reduced[:, 0, 0], X_reduced[0, :, 0], Z_reduced[0, 0, :]), vz_grid)        
        
        return vx_Interp([x[1], x[0], x[2]])[0], vy_Interp([x[1], x[0], x[2]])[0], vz_Interp([x[1], x[0], x[2]])[0]